In [1]:
import pandas as pd
import numpy as np

In [3]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])

In [4]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five  

1. 

In [10]:
temp = df2010.loc[:, ['retdate', 'vol']].groupby(df2010.loc[:, 'retdate'].dt.day_of_week, as_index= False).agg({'vol': 'sum'}).reset_index()
temp.columns = ['weekday', 'tot_vol']
temp.sort_values('tot_vol', ascending= False).iloc[[0],:]

,weekday,tot_vol
1,1,257616205528


2. 

In [13]:
temp = df2010.loc[:, ['retdate', 'vol']].groupby(df2010.loc[:, 'retdate'].dt.day_of_week, as_index= False).agg({'vol': 'mean'}).reset_index()
temp.columns = ['weekday', 'avg_vol']
temp.sort_values('avg_vol', ascending= False).iloc[[0],:]

,weekday,avg_vol
3,3,1.557712e+06


3.

In [23]:
temp = df2010.copy().assign(ret = df2010.loc[:, 'cls'] - df2010.loc[:, 'opn']).\
    loc[:, ['retdate', 'ret']].\
        groupby([df2010.loc[:, 'retdate'].dt.day_of_week, df2010.loc[:, 'retdate'].dt.month]).agg({'ret': 'mean'}).nlargest(1, 'ret')
temp

,,ret
retdate,retdate,
3,11,78.232966


4. 

In [38]:
temp = df2010.copy()
temp.loc[:, 'C2'] = None
temp.loc[(temp.loc[:, 'vol'].between(1000000, 2000000)), 'C2'] = temp.loc[:, 'vol']
temp.loc[: , 'C3'] = None
temp.loc[(temp.loc[:, 'vol'] < 1000000) | (temp.loc[:, 'vol'] > 2000000), 'C3'] = temp.loc[:, 'vol']
temp.groupby(temp.loc[:, 'retdate'].dt.strftime('%A')).agg({'C2': 'mean', 'C3': 'mean'}).reset_index()

,retdate,C2,C3
0,Friday,1.418570e+06,1.561169e+06
1,Monday,1.421753e+06,1.324039e+06
2,Thursday,1.423380e+06,1.568947e+06
3,Tuesday,1.422890e+06,1.539453e+06
4,Wednesday,1.422226e+06,1.519873e+06


5. 

In [40]:
df2010.groupby(df2010.loc[:, 'symb'], as_index= False).agg({'cls': ['max']}).sort_values(('cls', 'max'), ascending =  False)

,symb,cls
,,max
945,DRYS,11373600.00
457,BRK.A,125612.00
1019,EGLE,8625.60
2857,SQQQ,4926.72
3021,TOPS,1792.00
...,...,...
2449,PLUG,0.74
1921,MBRX,0.47
2982,TGEN,0.32


# Main Problems

1. 

In [45]:
df2010.groupby(df2010.loc[:, 'retdate'].dt.quarter).agg({'retdate': ['count']}).nlargest(1, ('retdate', 'count'))

,retdate
,count
retdate,
4,211456


2.

In [51]:
temp = df2010.groupby(df2010.loc[:, 'symb'], as_index=False).agg({'cls': ['max']})
temp.columns = ['symb', 'max_cls']
temp.loc[(temp.loc[:, 'max_cls'] > 100), 'DFlag'] = 1
temp.loc[(temp.loc[:, 'max_cls'] > 50) & (temp.loc[:, 'max_cls'] < 100), 'DFlag'] = 2
temp.loc[(temp.loc[:, 'max_cls'] < 50), 'DFlag'] = 3
temp

,symb,max_cls,DFlag
0,A,29.9857,3.0
1,AA,52.3500,2.0
2,AAME,2.0500,3.0
3,AAN,23.8000,3.0
4,AAON,8.6697,3.0
...,...,...,...
3357,ZIXI,4.4900,3.0
3358,ZN,7.2000,3.0
3359,ZNH,38.6500,3.0
3360,ZTR,15.8400,3.0


3. 

In [53]:
temp = df2010.groupby(df2010.loc[:, 'symb'], as_index=False).agg({'cls': ['max']})
temp.columns = ['symb', 'max_cls']
temp.loc[(temp.loc[:, 'max_cls'] > 100), 'DFlag'] = 1
temp.loc[(temp.loc[:, 'max_cls'] > 50) & (temp.loc[:, 'max_cls'] < 100), 'DFlag'] = 2
temp.loc[(temp.loc[:, 'max_cls'] < 50), 'DFlag'] = 3
temp.groupby(temp.loc[:, 'DFlag'], as_index = False).agg({'symb': ['nunique']})

,DFlag,symb
,,nunique
0,1.0,97
1,2.0,403
2,3.0,2862


4. 

In [59]:
temp = df2010.groupby(df2010.loc[:, 'symb'], as_index=False).agg({'cls': ['max']})
temp.columns = ['symb', 'max_cls']
temp.loc[(temp.loc[:, 'max_cls'] > 100), 'DFlag'] = 1
temp.loc[(temp.loc[:, 'max_cls'] > 50) & (temp.loc[:, 'max_cls'] < 100), 'DFlag'] = 2
temp.loc[(temp.loc[:, 'max_cls'] < 50), 'DFlag'] = 3
out = pd.DataFrame({'above100': (temp.loc[(temp.loc[:, 'DFlag'] == 1), 'symb'].nunique()),
                    'between': (temp.loc[(temp.loc[:, 'DFlag'] == 2), 'symb'].nunique()),
                    'below50': (temp.loc[(temp.loc[:, 'DFlag'] == 3), 'symb'].nunique())}, index = [1])
out

,above100,between,below50
1,97,403,2862


5. 

In [65]:
df2010.groupby(df2010.loc[:, 'retdate'].dt.strftime('%B')).agg({'retdate': ['nunique']}).reset_index()

retdate        
              nunique
0       April      21
1      August      22
2    December      22
3    February      19
4     January      19
5        July      21
6        June      22
7       March      23
8         May      20
9    November      21
10    October      21
11  September      21

6. 

In [73]:
temp = df2010.loc[(df2010.loc[:, 'retdate'].dt.month == 12), :].groupby(df2010.loc[:, 'symb']).agg({'retdate': ['nunique'],
                                                                                                    'cls': ['max', 'min']}).reset_index()
temp.columns = ['symb', 'n_obs', 'max_cls', 'min_cls']
temp.loc[(temp.loc[:, 'n_obs'] == 22), :].assign(diff = temp.loc[:, 'max_cls'] - temp.loc[:, 'min_cls'])

,symb,n_obs,max_cls,min_cls,diff
0,A,22,29.9857,25.8369,4.1488
1,AA,22,46.1700,40.7100,5.4600
2,AAME,22,2.0500,1.9300,0.1200
3,AAN,22,20.8100,19.9500,0.8600
4,AAON,22,8.6697,7.9022,0.7675
...,...,...,...,...,...
3329,ZIXI,22,4.4900,3.7100,0.7800
3330,ZN,22,4.9300,4.5700,0.3600
3331,ZNH,22,36.1800,28.1800,8.0000
3332,ZTR,22,14.8000,14.0000,0.8000


7. 

In [89]:
df2010.loc[(df2010.loc[:, 'retdate'].dt.day_of_week == 1), ['retdate']].assign(day = df2010.loc[:, 'retdate'].dt.strftime('%A'))

,retdate,day
1411,2010-04-13,Tuesday
1412,2010-04-13,Tuesday
1413,2010-04-13,Tuesday
1414,2010-04-13,Tuesday
1415,2010-04-13,Tuesday
...,...,...
812458,2010-12-07,Tuesday
812459,2010-12-07,Tuesday
812460,2010-12-07,Tuesday
812461,2010-12-07,Tuesday


In [90]:
temp = df2010.loc[(df2010.loc[:, 'retdate'].dt.month == 1) &
                   (df2010.loc[:, 'retdate'].dt.day_of_week == 1) &
                     (df2010.loc[:, 'exch'] == 'NYSE'), :].groupby(df2010.loc[:, 'symb']).agg({'retdate': ['nunique'],
                                                                                                    'cls': ['max', 'min']}).reset_index()
temp.columns = ['symb', 'n_obs', 'max_cls', 'min_cls']
temp.loc[(temp.loc[:, 'n_obs'] == 4), :].assign(diff = temp.loc[:, 'max_cls'] - temp.loc[:, 'min_cls']).agg({'diff': ['mean']})

,diff
mean,3.462797


8. 

In [78]:
temp = df2010.loc[(df2010.loc[:, 'retdate'].dt.month == 1) &
                   (df2010.loc[:, 'retdate'].dt.day_of_week == 1) &
                     (df2010.loc[:, 'exch'] == 'NYSE'), :].groupby(df2010.loc[:, 'symb']).agg({'retdate': ['nunique'],
                                                                                                    'cls': ['mean']}).reset_index()
temp.columns = ['symb', 'n_obs', 'mean_cls']
temp.loc[(temp.loc[:, 'n_obs'] == 4), :].agg({'mean_cls': ['mean']})

,mean_cls
mean,86.794097


9. 

In [81]:
symb_2011 = df2011.loc[(df2011.loc[:, 'cls'] > 100), 'symb'].drop_duplicates()
df2010.loc[(df2010.loc[:, 'symb'].isin(symb_2011)) & (df2010.loc[:, 'exch'] == 'NYSE'), :].groupby(df2010.loc[:, 'retdate'].dt.strftime('%B')).agg({'cls': ['mean']}).reset_index()

,retdate,cls
,,mean
0,April,1924.819517
1,August,1866.078577
2,December,1932.185474
3,February,1858.233541
4,January,1686.006488
5,July,1862.405672
6,June,1818.482760
7,March,1995.268784
8,May,1804.254212


10. 

In [83]:
max_price = df2010.loc[:, 'cls'].max()
df2010.loc[(df2010.loc[:, 'cls'] < .001*max_price) &
                     (df2010.loc[:, 'exch'] == 'NYSE'), 'cls'].mean()

29.546883068779575